## Import Libraries

In [90]:
import numpy as np
from matplotlib import image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from random import randrange
import glob


## Load Images

In [62]:
def random_crop(img, width):
    x, y = img.size
    matrix = width

    x1 = randrange(0, x - matrix)
    y1 = randrange(0, y - matrix)
    return img.crop((x1, y1, x1 + matrix, y1 + matrix))

In [63]:
def load_images(folderpath, limit, img_size_crop):
    image_paths = sorted(glob.glob(folderpath))
    img_arr = []
    for img_path in image_paths[:limit]:
        print('Loading', img_path)
        image = Image.open(img_path).convert('L')
#         image = random_crop(image, img_size_crop)
        image = np.asarray(image)
        # summarize shape of the pixel array
        img_arr.append(image)
    
    return img_arr

In [64]:
hr_img_arr = load_images("../dataset/DIV2K_valid_HR/*.png", 10, 10)

Loading ../dataset/DIV2K_valid_HR/0801.png
Loading ../dataset/DIV2K_valid_HR/0802.png
Loading ../dataset/DIV2K_valid_HR/0803.png
Loading ../dataset/DIV2K_valid_HR/0804.png
Loading ../dataset/DIV2K_valid_HR/0805.png
Loading ../dataset/DIV2K_valid_HR/0806.png
Loading ../dataset/DIV2K_valid_HR/0807.png
Loading ../dataset/DIV2K_valid_HR/0808.png
Loading ../dataset/DIV2K_valid_HR/0809.png
Loading ../dataset/DIV2K_valid_HR/0810.png


In [65]:
lr_img_arr = load_images("../dataset/DIV2K_valid_LR_bicubic/x2/*.png", 10, 10)

Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0801x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0802x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0803x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0804x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0805x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0806x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0807x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0808x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0809x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0810x2.png


In [66]:
lr_img_arr = [e.flatten() for e in lr_img_arr]

In [67]:
len(lr_img_arr)

10

## Downsample and Blur HR Images

In [68]:
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

In [69]:
import sys
sys.path.append('..')

In [70]:
from python_pipeline.obs_for_SR import obs_for_SR

Run downsampling + blur on all images

In [71]:
transformed_lr_img_arr = np.array([
    obs_for_SR(
        image, 
        matlab_style_gauss2D((7,7), 2),
        1, 
        2
    ).flatten() for image in hr_img_arr
])

<ipython-input-71-7aece411d74e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  transformed_lr_img_arr = np.array([


In [72]:
len(transformed_lr_img_arr)

10

## Gaussian Distribution

In [74]:
from scipy.stats import multivariate_normal as mvn

In [93]:
# Work with one image
def gauss_pdf(lr_img, transformed_lr_img):
    mu = np.mean(transformed_lr_img)
    std = np.std(transformed_lr_img)
    var = np.var(transformed_lr_img)
    return mvn.pdf(lr_img, mean=mu, cov=var)


In [96]:
gauss_pdf(lr_img_arr[0], transformed_lr_img_arr[0])

array([0.00224763, 0.00301227, 0.00386395, ..., 0.00167186, 0.00182385,
       0.00270458])